# 1. PyGMT Refresher Webinar

# 1.1 Learning Objectives

- **1.2**: Provide an overview of PyGMT
- **2.1**: Load x,y,z data into a pandas.DataFrame
- **2.2**: Plot tabular data
- **2.3**: Grid x,y,z data
- **2.4**: Plot gridded data
- **3.1**: Add map embellishments
- **4.1**: Add insets
- **5.1**: Create subplots


## 1.2 PyGMT Overview

### What is PyGMT?

- PyGMT is an open-source Python package for geospatial data processing, analysis, and visualization.
- PyGMT is designed to:
  - support rich display in Jupyter notebooks.
  - improve access to the Generic Mapping Tools (GMT).
  - integrate smoothly with scientific Python packages (e.g., NumPy, pandas, xarray, GeoPandas).
  
### How does PyGMT compare to other common Python packages and software?

**How does PyGMT differ from GMT?**

- PyGMT provides a Python interface for the GMT C API, which has been traditionally accessed
  through the command line.

**How does PyGMT differ from Matplotlib?**

- PyGMT produces static vector graphics, whereas Matplotlib has a larger focus on
  interactivity.

**How does PyGMT differ from Cartopy?**

- PyGMT is built on GMT's processing and plotting functionality, whereas Cartopy is built on
  Matplotlib's plotting functionality, along with NumPy, Shapely, and PROJ.4.
  
### Where is the documentation for PyGMT?

- The documentation is located at https://www.pygmt.org/latest/index.html.
- To view the documentation for a different version, use the drop-down menu
  in the upper-left corner or the links in the
  [compatibility table](https://www.pygmt.org/latest/index.html#compatibility-with-gmt-python-numpy-versions).
  Since this refresher is associated with a developer workshop, all the links in
  the tutorial will go to the [dev docs](https://www.pygmt.org/dev/index.html).
- If you are working in a Jupyter Notebook, you can also view the docstrings for
  PyGMT functions by clicking shift-tab after typing in the function name (and
  you can use tab-completion to finish function names).

### Warning about PyGMT

PyGMT is undergoing rapid development. This makes now a great time to get involved with the
PyGMT Community to help shape the future of the project. It also means that all of the
application program interface (API; i.e., how you interact with PyGMT), is subject to change
with non-backward compatible changes. Non-backward compatible means that the workflows
that you create today are not guaranteed to work in the future, although the PyGMT
[deprecation policy](https://www.pygmt.org/dev/maintenance.html#backwards-compatibility-and-deprecation-policy)
includes the goals of providing warning about these changes and only making necessary
changes.

## 2.1: Load x,y,z data into a pandas.DataFrame

PyGMT excels at processing and plotting geographic data. The first few sections of this
tutorial cover some processing capabilities of PyGMT while the last sections cover
plotting capabilities. First, let's get started by loading in some sample data using
one of PyGMT's [functions to load sample datasets](https://www.pygmt.org/dev/api/index.html#datasets).

In [ ]:
import pygmt

In [ ]:
# Load a table of ship bathymetric observations off Baja California
data = pygmt.datasets.load_sample_bathymetry()
# Return a summary about the dataset using pandas.DataFrame.info()
data.info()

In [ ]:
# Use pygmt.info() to store the x/y range of data values rounded to the nearest degree
region = pygmt.info(data, spacing=1)
region

## 2.2 Plot tabular data

Let's plot the x,y,z data! The basic steps for plotting in
PyGMT are to create an instance of the Figure class and
the call its plotting methods. The first time that you
call a plotting method, you will probably want to use
these three parameters:

- The ``frame`` parameter. 
  - This parameter controls the appearance of the map frame
    boundary.
  - For simple, automatic frame settings you can use ``frame=True``.
  - There's a lot the frame parameter can do. We'll learn more
    later, or you can check out the [frame tutorial](https://www.pygmt.org/dev/tutorials/frames.html).
- The ``region`` parameter.
  - This parameter controls the boundaries of the plot, usually
    by accepting an array in the form `[xmin, xmax, ymin, ymax]`.
  - There are a few special options for the ``region`` parameter,
    which you can learn about in the [region tutorial](https://www.pygmt.org/dev/tutorials/regions.html).
- The ``projection`` parameter.
  - This parameter controls how your data are mapped onto a 2D
    plot. The control over projection is what sets libraries
    like PyGMT and Cartopy apart from libraries focuses on
    non-geographic datasets. There are many projections
    options listed in the [projections table](https://www.pygmt.org/dev/projections/index.html#projection-table)
    with examples shown in the [projections gallery](https://www.pygmt.org/dev/projections/index.html).

    

In [ ]:
# Make a quick plot of the data
# Create an instance of the pygmt.Figure() class
fig = pygmt.Figure()
# Plot the data on the figure using 1 point, blue circles (style="c1p", color="slateblue")
# Set a simple, automatic frame (``frame=True``)
# Use a Mercator projection on a 6 inch wide plot (``projection="M6i"``)
fig.plot(data=data, frame=True, region=region, projection="M6i", style="c1p", color="slateblue")
# Display the figure
fig.show()

## 2.3 Grid x,y,z data

There are many different ways to grid irregularly-spaced x,y,z data. Here we'll use
one workflow for gridding tabular data using PyGMT:

- Preprocess data using moving median to avoid spatial aliasing and eliminate redundant data
- Use a minimum curvature technique to produce gridded values from x,y,z triplets


### Block average data using median estimation

In [ ]:
# Specify that 10 arc-minute blocks will be used for the analysis
spacing="10m"
# Compute the median position and value for every non-empty 10 arc-minute block
# in the bathymetry dataset
data_median = pygmt.blockmedian(data, region=region, spacing=spacing)
# Return a summary about the dataset using pandas.DataFrame.info()
data_median.info()

### Plot the x,y,z data returned from blockmedian

In [ ]:
# Create an instance of the pygmt.Figure() class
fig = pygmt.Figure()
# Plot the data on the figure using 5 point, blue circles (style="c")
# Use a Mercator projection on a 6 inch wide plot (``projection="M6i"``)
# Use automatic annotation and tick intervals and set a title (``frame=["af", "+t"Results from pygmt.blockmedian()""]``)
fig.plot(data=data_median, frame=["af", '+t"Results from pygmt.blockmedian()"'], region=region, projection="M6i", style="c5p", color="slateblue")
# Display the figure
fig.show()

### Grid the data using continuous curvature splines

In [ ]:
# First, we need to convert the pandas.DataFrame to a numpy ndarray
# (most other functions that accept tabular input accept dataframes)
# (see https://github.com/GenericMappingTools/pygmt/issues/1443 for details)
data_median = data_median.to_numpy()

# Produce a grid from the x,y,z data using pygmt.surface()
grid = pygmt.surface(data=data_median, region=region, spacing=spacing)
# Inspect the xarray.dataarray returned from pygmt.surface()
grid

## 2.4 Plot gridded data

One of the most common methods to plot gridded data with
PyGMT is using the Figure.grdimage method. As with before,
we'll use the region, projection, and frame parameters to
control the map appearance. We'll also use the
pygmt.makecpt function to create a colormap using one of the
[Scientific Colour Maps](https://www.fabiocrameri.ch/colourmaps/)
available through GMT.

In [ ]:
# Create an instance of the pygmt.Figure() class
fig = pygmt.Figure()
# Create a colormap for the data using pygmt.makecpt()
# Use the `lapaz` scientific colormap (cmap=`lapaz`)
pygmt.makecpt(cmap="lapaz", series=[-8000, 0])
# Plot the data on the figure using 5 point, blue circles (style="c")
# Use a Mercator projection on a 6 inch wide plot (``projection="M6i"``)
# Use automatic annotation and tick intervals and set a title (``frame=["af", "+t"Results from pygmt.blockmedian()""]``)
fig.grdimage(grid=grid, frame=["af", '+t"Results from pygmt.blockmedian()"'], region=region, projection="M6i", shading=True)
# Overlay contours from the gridded data
fig.grdcontour(grid=grid, limit=[-8000, 0], interval=250, annotation=1000)
# Display the figure
fig.show()